# Notebook 01: Data Exploration (for Edge-IIoT)

**Goal:** Establish a factual understanding of the raw dataset that will feed the FL-IDS for IIoT (surveillance). This notebook inspects structure, label balance, and data quality. It does not modify or export data.

**Objectives**
- Load the raw CSV from `data/raw/...`.
- Inspect shape, columns, dtypes, and memory footprint.
- Check class balance for `Attack_label` and enumerate `Attack_type`.
- Identify potential data quality issues (missingness, mixed types, high-cardinality text).
- List actions to perform later in data preparation.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 1. Dataset Loading and Structure

This section confirms that the expected file is present and examines high-level structure:

- File path used in this run (printed above).
- Dataset shape (rows × columns).
- A plaintext list of all columns to anchor later selections.
- Memory footprint will be captured via `df.info()` below.

**Notes**
- Keep `low_memory=False` to avoid misleading dtypes on wide CSVs.
- If the file is missing, place it under `data/raw/` and re-run.


In [2]:
#Defining the paths and loading the dataset 
file_path = r"D:\August-Thesis\FL-IDS-Surveillance\data\raw\archive\DNN-EdgeIIoT-dataset.csv"
df = pd.read_csv(file_path, low_memory = False)

#Display Preliminary info about the dataset
print (f"The shape of the dataset: {df.shape}")
print ("The columns of the dataset are: \n", df.columns.tolist())


The shape of the dataset: (2219201, 63)
The columns of the dataset are: 
 ['frame.time', 'ip.src_host', 'ip.dst_host', 'arp.dst.proto_ipv4', 'arp.opcode', 'arp.hw.size', 'arp.src.proto_ipv4', 'icmp.checksum', 'icmp.seq_le', 'icmp.transmit_timestamp', 'icmp.unused', 'http.file_data', 'http.content_length', 'http.request.uri.query', 'http.request.method', 'http.referer', 'http.request.full_uri', 'http.request.version', 'http.response', 'http.tls_port', 'tcp.ack', 'tcp.ack_raw', 'tcp.checksum', 'tcp.connection.fin', 'tcp.connection.rst', 'tcp.connection.syn', 'tcp.connection.synack', 'tcp.dstport', 'tcp.flags', 'tcp.flags.ack', 'tcp.len', 'tcp.options', 'tcp.payload', 'tcp.seq', 'tcp.srcport', 'udp.port', 'udp.stream', 'udp.time_delta', 'dns.qry.name', 'dns.qry.name.len', 'dns.qry.qu', 'dns.qry.type', 'dns.retransmission', 'dns.retransmit_request', 'dns.retransmit_request_in', 'mqtt.conack.flags', 'mqtt.conflag.cleansess', 'mqtt.conflags', 'mqtt.hdrflags', 'mqtt.len', 'mqtt.msg_decoded_as

## 2. Class Distribution and Attack Types

We summarise label balance for:
- `Attack_label` (binary target: 0 = normal, 1 = attack).
- `Attack_type` (multi-class taxonomy of specific attacks).

**Why this matters**
- Guides metrics and sampling strategies.
- Informs whether we need stratified splits and per-class reporting.

Record both counts and percentages, and note any rare classes that may affect evaluation.


In [6]:
# Checking the unique values in "Attack_type" & "Attack_label" to see the type of the attacks this dataset cover
attack_types  = df["Attack_type"].value_counts()
attack_labels = df["Attack_label"].value_counts()

attack_types , attack_labels

(Attack_type
 Normal                   1615643
 DDoS_UDP                  121568
 DDoS_ICMP                 116436
 SQL_injection              51203
 Password                   50153
 Vulnerability_scanner      50110
 DDoS_TCP                   50062
 DDoS_HTTP                  49911
 Uploading                  37634
 Backdoor                   24862
 Port_Scanning              22564
 XSS                        15915
 Ransomware                 10925
 MITM                        1214
 Fingerprinting              1001
 Name: count, dtype: int64,
 Attack_label
 0    1615643
 1     603558
 Name: count, dtype: int64)

## 3. Data Types, Missingness, and Potential Quality Risks

We capture:
- Full dtype table from `df.info()`.
- Column-wise missing value counts.
- Early flags for columns with mixed or ambiguous types.

**Interpretation checklist**
- Columns expected to be numeric but parsed as `object` (e.g., ports, lengths).
- High-cardinality text fields (IPs, URIs, topics) that are unsuitable for standard models without encoding or pruning.
- Columns that may be redundant or leak labels.


In [7]:
#Checking the data types and missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2219201 entries, 0 to 2219200
Data columns (total 63 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   frame.time                 object 
 1   ip.src_host                object 
 2   ip.dst_host                object 
 3   arp.dst.proto_ipv4         object 
 4   arp.opcode                 float64
 5   arp.hw.size                float64
 6   arp.src.proto_ipv4         object 
 7   icmp.checksum              float64
 8   icmp.seq_le                float64
 9   icmp.transmit_timestamp    float64
 10  icmp.unused                float64
 11  http.file_data             object 
 12  http.content_length        float64
 13  http.request.uri.query     object 
 14  http.request.method        object 
 15  http.referer               object 
 16  http.request.full_uri      object 
 17  http.request.version       object 
 18  http.response              float64
 19  http.tls_port              float64
 20  tc

In [8]:
# cOUNT of missing values for each column in the dataset
missing = df.isnull().sum()
missing[missing > 0].sort_values(ascending = False)

Series([], dtype: int64)

## 4. Categorical and Protocol Fields — Sanity Checks

We inspect uniqueness samples for selected fields:
- `http.request.method`, `http.request.version`
- `tcp.srcport`
- `dns.qry.name.len`

**What to look for**
- Unexpected tokens embedded in protocol/version fields (e.g., payload fragments or injections).
- Numeric-looking fields stored as strings.
- Parsing artefacts that will require cleaning later.

No transformations are performed here; this is an audit to inform preparation steps.


In [10]:
#Check the unique values of potential categorical fiedls such as the http request method and version
unique_http_method = df["http.request.method"].unique()
unique_http_version = df["http.request.version"].unique()
unique_tcpport = df["tcp.srcport"].unique()[:10] #first ten should be enough to get an idea if it is numeric
unique_dns_namelen = df["dns.qry.name.len"].unique()[:10] #first ten should be enough to get an idea if it is numeric

print(f"Http.request.method: {unique_http_method}")
print(f"Http.request.version : {unique_http_version}")
print(f"tcp.srcport sample: {unique_tcpport}")
print(f"dns.qry.name.len sample: {unique_dns_namelen}")

Http.request.method: ['0.0' '0' 'GET' 'POST' 'PROPFIND' 'TRACE' 'OPTIONS' 'PUT' 'SEARCH']
Http.request.version : ['0.0' '0' 'HTTP/1.1' 'HTTP/1.0'
 "name=a><input name=i value=XSS>&lt;script>alert('Vulnerable')</script> HTTP/1.1"
 'Src=javascript:alert(\'Vulnerable\')><Img Src=\\" HTTP/1.1' '> HTTP/1.1'
 'script>alert(1)/script><\\" HTTP/1.1'
 '-al&_PHPLIB[libdir]=http://cirt.net/rfiinc.txt?? HTTP/1.1' '-a HTTP/1.1'
 '/etc/passwd|?data=Download HTTP/1.1'
 '-al&ABSOLUTE_PATH_STUDIP=http://cirt.net/rfiinc.txt?? HTTP/1.1'
 'By Dr HTTP/1.1']
tcp.srcport sample: ['1883.0' '64855.0' '64856.0' '64857.0' '64858.0' '64859.0' '64860.0'
 '64861.0' '64862.0' '64863.0']
dns.qry.name.len sample: ['0' '0.debian.pool.ntp.org' '1.debian.pool.ntp.org'
 '2.debian.pool.ntp.org' '3.debian.pool.ntp.org' '_googlecast._tcp.local'
 'raspberrypi.local' 'null-null.local' '0.0' '1.0']


## 5. Notes and Next Steps

**Key EDA findings**
- Summarise any missing values, suspicious dtypes, and extreme cardinality fields identified above.
- List columns that look text-heavy or noisy and likely to be dropped or transformed later.

**Next steps**
- Column selection and safe casting plan.
- Handling of high-cardinality text (e.g., IPs, URIs, MQTT topics).
- Definitions of train/test sets for supervised and unsupervised pipelines.
- Reproducibility choices (random seeds) and consistent, repo-relative paths.

The actual cleaning, splitting, and exports are performed in the subsequent section(s) to keep this EDA record clear and auditable.


## 6. Cleaning the Dataset for Binary Classification

This step standardises the raw Edge-IIoT data for downstream supervised and unsupervised pipelines—without changing labels.

**Rationale**
- Drop high-cardinality / unstructured fields (IPs, URIs, raw payloads, free-text) that are not robust for our models.
- Ensure key network fields are in numeric form (e.g., `tcp.srcport`).
- Use `Attack_label` as the binary target and remove `Attack_type` (multi-class taxonomy not used in binary models).

**Operations performed**
- Drop columns: IPs/URIs/payloads and related free-text protocol fields (see code list below for exact names).
- Convert `tcp.srcport` to numeric (coercing errors), drop rows where it is invalid/NaN, then cast to integer.
- Retain only modeling-relevant features + `Attack_label`.

**Outputs**
- Cleaned CSV for supervised work.
- Identical cleaned CSV for unsupervised work.


### Cleaned Data Artifacts (for later use)

Saved cleaned datasets for both tracks:

- **Supervised:** `data/processed/surv_supervised/surv_cleaned_supervised.csv`
- **Unsupervised:** `data/processed/surv_unsupervised/surv_cleaned_unsupervised.csv`

**Note**
- Keep these paths repo-relative in code where possible.
- These files are large and should remain git-ignored (`data/**`), but paths are documented here for reproducibility.


In [13]:
#Columns to drop
columns_to_drop = [
    "ip.src_host", "ip.dst_host", "arp.dst.proto_ipv4", "arp.src.proto_ipv4","http.file_data", "http.request.uri.query",
    "http.request.full_uri", "http.referer","dns.qry.name", "dns.qry.name.len", "mqtt.topic", "mqtt.msg", 
    "mqtt.msg_decoded_as","mqtt.protoname", "tcp.options", "tcp.payload", "mqtt.conack.flags",
    "http.request.version", "frame.time", "Attack_type"
]

#Drop the columns
df_cleaned = df.drop(columns = columns_to_drop)

# Convert tcp.srcport to numeric, coerce errors to NaNs
df_cleaned['tcp.srcport'] = pd.to_numeric(df_cleaned['tcp.srcport'], errors='coerce')

#Double Checking:
print(f"NaNs in tcp.srcport: {df_cleaned['tcp.srcport'].isna().sum()}")

# Drop rows where tcp.srcport is NaN
df_cleaned.dropna(subset=['tcp.srcport'], inplace=True)
# Convert to integer now that NaNs are gone
df_cleaned['tcp.srcport'] = df_cleaned['tcp.srcport'].astype(int)

print(f"The Cleaned dataset shape after dropping the invalid tcp.srcport rows: {df_cleaned.shape}")

# Saving the cleaned dataset to both supervised and unsupervised folders
output_supervised = r"D:\August-Thesis\FL-IDS-Surveillance\data\processed\surv_supervised\surv_cleaned_supervised.csv"
output_unsupervised = r"D:\August-Thesis\FL-IDS-Surveillance\data\processed\surv_unsupervised\surv_cleaned_unsupervised.csv"

#Save the dataset (same file for both for now)
df_cleaned.to_csv(output_supervised, index=False)
df_cleaned.to_csv(output_unsupervised, index=False)
print("Cleaned dataset saved to:")
print(f"Supervised: {output_supervised}")
print(f"Unsupervised: {output_unsupervised}")


NaNs in tcp.srcport: 367
The Cleaned dataset shape after dropping the invalid tcp.srcport rows: (2218834, 43)
Cleaned dataset saved to:
Supervised: D:\August-Thesis\FL-IDS-Surveillance\data\processed\surv_supervised\surv_cleaned_supervised.csv
Unsupervised: D:\August-Thesis\FL-IDS-Surveillance\data\processed\surv_unsupervised\surv_cleaned_unsupervised.csv


## 8. Supervised 80/20 Split (with and without SMOTE)

We create two supervised training variants from the cleaned dataset:

1) **No-SMOTE:** baseline stratified 80/20 split.  
2) **With-SMOTE:** same split, then apply SMOTE **on the training set only** to reduce class imbalance.

**Procedure**
- Read `surv_cleaned_supervised.csv`.
- Separate features/labels (`Attack_label`).
- Perform **stratified** `train_test_split` (80/20, `random_state=42`).
- If present, label-encode `http.request.method` (a small categorical field) to maintain numeric design.
- Save both variants for later use.

**Saved outputs**
- `data/processed/surv_supervised/80_20/no_smote/train.csv`  
- `data/processed/surv_supervised/80_20/no_smote/test.csv`  
- `data/processed/surv_supervised/80_20/with_smote/train.csv`  
- `data/processed/surv_supervised/80_20/with_smote/test.csv`  *(test unchanged)*


In [14]:
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
import os
from sklearn.model_selection import train_test_split

cleaned_path = r"D:\August-Thesis\FL-IDS-Surveillance\data\processed\surv_supervised\surv_cleaned_supervised.csv"
df_cleaned = pd.read_csv(cleaned_path, low_memory = False)

#Seperate the features from labels
X = df_cleaned.drop(columns=['Attack_label'])
y = df_cleaned['Attack_label']

# 80/20 Split Stratified Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Label encode 'http.request.method' if it exists
if 'http.request.method' in X_train.columns:
    le = LabelEncoder()
    X_train['http.request.method'] = le.fit_transform(X_train['http.request.method'])
    X_test['http.request.method'] = le.transform(X_test['http.request.method'])

# Save non-SMOTE version
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

no_smote_path = r"D:\August-Thesis\FL-IDS-Surveillance\data\processed\surv_supervised\80_20\no_smote"

os.makedirs(no_smote_path, exist_ok=True)
train_df.to_csv(os.path.join(no_smote_path, "train.csv"), index=False)
test_df.to_csv(os.path.join(no_smote_path, "test.csv"), index=False)
print("Saved 80/20 split without SMOTE.")

# === SMOTE version ===
sm = SMOTE(random_state=42)
X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)
train_smote_df = pd.concat(
    [pd.DataFrame(X_train_sm, columns=X_train.columns),
      pd.Series(y_train_sm, name='Attack_label')], axis=1
 )

smote_path = r"D:\August-Thesis\FL-IDS-Surveillance\data\processed\surv_supervised\80_20\with_smote"
os.makedirs(smote_path, exist_ok=True)
train_smote_df.to_csv(os.path.join(smote_path, "train.csv"), index=False)
test_df.to_csv(os.path.join(smote_path, "test.csv"), index=False) # unchanged
print("Saved 80/20 split with SMOTE applied to training set.")

Saved 80/20 split without SMOTE.
Saved 80/20 split with SMOTE applied to training set.


## 9. Preparing Unsupervised Anomaly-Detection Sets

Unsupervised models (Isolation Forest, Autoencoder) learn only from **normal** traffic and are evaluated on mixed traffic.

**Procedure**
- Read `surv_cleaned_unsupervised.csv`.
- **Training set:** all rows where `Attack_label == 0` (normal-only).
- **Testing set:** full cleaned dataset (normal + attack).
- Save both artifacts for downstream unsupervised modeling.

**Saved outputs**
- `data/processed/surv_unsupervised/train_normal_only.csv`
- `data/processed/surv_unsupervised/test_mixed.csv`


In [16]:
# Load the already-cleaned dataset
cleaned_path = r"D:\August-Thesis\FL-IDS-Surveillance\data\processed\surv_unsupervised\surv_cleaned_unsupervised.csv"
df_cleaned = pd.read_csv(cleaned_path, low_memory = False)

# Training set: only normal data (Attack_label == 0)
df_train_unsup = df_cleaned[df_cleaned['Attack_label'] == 0]

# Testing set: full data (mixed normal + attack)
df_test_unsup = df_cleaned.copy()

# Save both files
output_path = r"D:\August-Thesis\FL-IDS-Surveillance\data\processed\surv_unsupervised"
os.makedirs(output_path, exist_ok=True)
df_train_unsup.to_csv(os.path.join(output_path, "train_normal_only.csv"), index = False)
df_test_unsup.to_csv(os.path.join(output_path, "test_mixed.csv"), index = False)
 
print("Unsupervised dataset prepared and saved:")
print(f"- Training set (normal only): {df_train_unsup.shape}")
print(f"- Testing set (mixed): {df_test_unsup.shape}")

Unsupervised dataset prepared and saved:
- Training set (normal only): (1615643, 43)
- Testing set (mixed): (2218834, 43)


In [ ]:
import pandas as pd
import os
from sklearn.utils import shuffle

#Load the full normal-only dataset
path = r"D:\August-Thesis\FL-IDS-Surveillance\data\processed\surv_unsupervised\train_normal_only.csv"
df = pd.read_csv(path)
print("Original shape:", df.shape)

#Dropping the problematic columns
problem_cols = ['http.request.method']
df.drop(columns=[col for col in problem_cols if col in df.columns], inplace=True)

#Check the NaNs
print("Missing values before cleaning:\n", df.isnull().sum().sort_values(ascending=False).head())

# Drop rows with any NaNs
df.dropna(inplace=True)
print("Shape after dropping NaNs:", df.shape)

#Shuffle the dataset before the split
df = shuffle(df, random_state=42).reset_index(drop=True)

#Split into 5 equal parts
n_clients = 5
chunk_size = len(df) // n_clients

output_dir = r"D:\August-Thesis\FL-IDS-Surveillance\data\processed\federated\unsupervised"
os.makedirs(output_dir, exist_ok=True)

for i in range(n_clients):
    start = i * chunk_size
    end = (i + 1) * chunk_size if i < n_clients - 1 else len(df)
    df_client = df.iloc[start:end].copy()
    client_path = os.path.join(output_dir, f"client_{i+1}")
    os.makedirs(client_path, exist_ok=True)
    df_client.to_csv(os.path.join(client_path, "train.csv"), index=False)
    print(f"Client {i+1}: {df_client.shape[0]} rows saved to {client_path}/train.csv")


Original shape: (1615643, 43)
Missing values before cleaning:
 arp.opcode                 0
arp.hw.size                0
icmp.checksum              0
icmp.seq_le                0
icmp.transmit_timestamp    0
dtype: int64
Shape after dropping NaNs: (1615643, 42)
Client 1: 323128 rows saved to D:\August-Thesis\FL-IDS-Surveillance\data\processed\federated\unsupervised\client_1/train.csv
Client 2: 323128 rows saved to D:\August-Thesis\FL-IDS-Surveillance\data\processed\federated\unsupervised\client_2/train.csv
Client 3: 323128 rows saved to D:\August-Thesis\FL-IDS-Surveillance\data\processed\federated\unsupervised\client_3/train.csv
